In [7]:
import requests
import tarfile
from os import path

In [8]:
cd BigData

[WinError 2] The system cannot find the file specified: 'BigData'
f:\CodingEnvironment\BigData


In [11]:
!set HADOOP_HOME=F:\CodingEnvironment\hadoop-3.4.1-src
!set SPARK_HOME=F:\CodingEnvironment\spark-3.5.3-bin-hadoop3
!set PATH=%PATH%;%SPARK_HOME%\bin;%HADOOP_HOME%\bin

In [1]:
!echo %HADOOP_HOME%

%HADOOP_HOME%


In [2]:
!java --version

The system cannot find the file C:\ProgramData\Oracle\Java\javapath\java.exe.


'ls' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import requests
fname = 'airline.tar.gz'
url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/' + fname
r = requests.get(url)
open(fname , 'wb').write(r.content)

In [ ]:
tar = tarfile.open(fname)
tar.extractall()
tar.close()

In [12]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .getOrCreate()

ModuleNotFoundError: No module named 'pyspark'

In [22]:
!docker ps

CONTAINER ID   IMAGE           COMMAND                  CREATED          STATUS          PORTS                                            NAMES
f541c465d56b   bitnami/spark   "/opt/bitnami/script…"   18 minutes ago   Up 18 minutes   0.0.0.0:7077->7077/tcp, 0.0.0.0:8080->8080/tcp   spark-container


In [23]:
!telnet 192.168.4.58 7077

In [1]:
import pandas as pd

# Specify the path to your CSV file
data_path = 'F:\\CodingEnvironment\\BigData\\airline_2m.csv'

# Read only the first 5 rows of the file
df = pd.read_csv(
    data_path, 
    encoding="ISO-8859-1", 
    dtype={'Div1Airport': str, 'Div1TailNum': str, 'Div2Airport': str, 'Div2TailNum': str},
    nrows=5
)

# Preview the first 5 lines
print(df)


   Year  Quarter  Month  DayofMonth  DayOfWeek  FlightDate Reporting_Airline  \
0  1998        1      1           2          5  1998-01-02                NW   
1  2009        2      5          28          4  2009-05-28                FL   
2  2013        2      6          29          6  2013-06-29                MQ   
3  2010        3      8          31          2  2010-08-31                DL   
4  2006        1      1          15          7  2006-01-15                US   

   DOT_ID_Reporting_Airline IATA_CODE_Reporting_Airline Tail_Number  ...  \
0                     19386                          NW      N297US  ...   
1                     20437                          FL      N946AT  ...   
2                     20398                          MQ      N665MQ  ...   
3                     19790                          DL      N6705Y  ...   
4                     20355                          US      N504AU  ...   

   Div4WheelsOff  Div4TailNum  Div5Airport  Div5AirportID Div5

In [ ]:
pd.set_option('display.max_colwidth', None)  # No truncation for wide columns
pd.set_option('display.max_columns', None)

Year                2009
Quarter                2
Month                  5
DayofMonth            28
DayOfWeek              4
                    ... 
Div5WheelsOn         NaN
Div5TotalGTime       NaN
Div5LongestGTime     NaN
Div5WheelsOff        NaN
Div5TailNum          NaN
Name: 1, Length: 109, dtype: object


In [6]:
import pandas as pd

# Set the display options to show all columns
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)        # Ensure no line wrapping
pd.set_option('display.max_rows', None)     # Show all rows, if necessary

# Example: Print the first row
print(df.iloc[1])


Year                                        2009
Quarter                                        2
Month                                          5
DayofMonth                                    28
DayOfWeek                                      4
FlightDate                            2009-05-28
Reporting_Airline                             FL
DOT_ID_Reporting_Airline                   20437
IATA_CODE_Reporting_Airline                   FL
Tail_Number                               N946AT
Flight_Number_Reporting_Airline              671
OriginAirportID                            13342
OriginAirportSeqID                       1334202
OriginCityMarketID                         33342
Origin                                       MKE
OriginCityName                     Milwaukee, WI
OriginState                                   WI
OriginStateFips                               55
OriginStateName                        Wisconsin
OriginWac                                     45
DestAirportID       

In [1]:
import requests
import tarfile
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor
from google.cloud import storage

# Function to upload to GCS
def upload_to_gcs(bucket_name, blob_name, file_data):
    client = storage.Client()
    bucket = client.bucket(bucket_name)
    blob = bucket.blob(blob_name)
    blob.upload_from_file(file_data)

# Bucket name and URL
bucket_name = 'bk9999airline'  # Replace with your GCS bucket name
file_url = 'https://dax-cdn.cdn.appdomain.cloud/dax-airline/1.0.1/airline_2m.tar.gz'

# Step 1: Download the tar.gz file
r = requests.get(file_url)
r.raise_for_status()  # Ensure the request was successful
tarball_data = BytesIO(r.content)  # Convert to a byte stream

# Step 2: Extract the files from the tar.gz archive
with tarfile.open(fileobj=tarball_data, mode='r:gz') as tar:
    with ThreadPoolExecutor() as executor:
        # Step 3: Upload files to GCS in parallel
        for member in tar.getmembers():
            if member.isfile():  # Only process files (ignore directories)
                file_data = tar.extractfile(member)  # Get file data as a stream
                blob_name = member.name  # Use the member name for GCS path
                executor.submit(upload_to_gcs, bucket_name, blob_name, file_data)

print("Files are being uploaded to GCS.")

Files are being uploaded to GCS.
